In [43]:
%load_ext autoreload
%autoreload 2

In [244]:
from ExtractTable import ExtractTable
import pandas as pd
import numpy as np
import os
from io import BytesIO, BufferedReader
from PyPDF2 import PdfFileReader, PdfFileWriter
from mongoengine import register_connection, connect, disconnect
from pathlib import Path
from GC_beta.settings import BASE_DIR
from transcript_handler.models import University, Student, Department, Identifier, Transcript, Education
from transcript_handler.utils import extract_pages_from_raw_file, get_transcripts_and_dump_into_disk
from transcript_handler.handlers import ExtractTableHandler
import json
import fitz
from PIL import Image, ImageDraw

In [12]:
disconnect(alias='default')

In [7]:
register_connection(alias='default', name='GC_beta', host='localhost', port=27017)
connect(alias="default")

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

In [21]:
s1 = Student.objects.get(name='s1')

In [51]:
output_file_path = get_transcripts_and_dump_into_disk(s1, BASE_DIR)

C:\Projects\GC_beta\GC_beta\sample_output\s1\615f434184e36ad26ff8e075-[1,3].pdf


In [59]:
ExtractTableHandler.et_sess.check_usage()

{'credits': 3000, 'queued': 0, 'used': 11}

In [104]:
def compose_paths(student):
    paths = {}
    paths['input_file_path'] = os.path.join(BASE_DIR, "sample_output", student.name, f'{student.name}-raw-transcripts.pdf')
#     paths['input_file_path'] = f'{BASE_DIR}{INPUT_DIR}{category}\\{file_name}'
#     paths['output_dir'] = f'{BASE_DIR}{OUTPUT_DIR}{category}\\{file_name.split(".")[0]}\\'
    paths['output_dir'] = os.path.join(BASE_DIR, "sample_output", student.name)
    return paths

def dump_response(session, outdir):
    """
    dump table data(df as .csv) and raw server response(.json)
    """
    try:
        if not os.path.exists(outdir):
            os.mkdir(outdir)
        session.save_output(outdir, output_format="csv", indexing=True)
        with open(outdir+'\\server_response.json', 'w', encoding='utf-8') as f:
            json.dump(session.ServerResponse.json(), f, ensure_ascii=False, indent=4)
        return True
    except:
        return False
    
def read_response(outdir):
    tables = []  # dfs
    server_response = None  # json
    for f in os.listdir(outdir):
        if f.split('.')[1]=='csv':
            t = pd.read_csv(outdir+f)
            tables.append(t)
        elif f.split('.')[1]=='json':
            with open(outdir+f) as json_file:
                server_response = json.load(json_file)
    return tables, server_response

In [74]:
deng = Student.objects.get(name='Deng')

In [77]:
paths = compose_paths(deng)

In [78]:
table_data = ExtractTableHandler.et_sess.process_file(filepath=paths['input_file_path'], pages="all", output_format="df")

[Info]: Waiting to retrieve the output; JobId: 1a5ea0ef8d16defe5350f787c5b8849b841788487c4ca9df4f32d5967934c941


In [79]:
table_data

[           0      1                                         2               3  \
 0     Course   Code                              Course Title  Unit Attempted   
 1   Semester   1 of                                 2012-2013                   
 2       MATH   1110            Advanced Mathematics I (### I)            3.00   
 3       LANG   1011                         Chinese I (A#+ I)            3.00   
 4        ENG   1011                        English I (K#RX I)            3.00   
 5       I.T.   1010                    Information Technology            3.00   
 6       MATH   1070     Linear Algebra I (For FM Students) I)            3.00   
 7       P.E.   1010         Physical Education/Healthy Living            3.00   
 8        WPE   1010                           University Life            1.00   
 9   Semester                                    Total BAPTIST           19.00   
 10  Semester                  Grade Point Average = 2.88 KONG                   
 11    Course   

In [105]:
dump_response(ExtractTableHandler.et_sess, paths['output_dir'])

True

In [95]:
jd = ExtractTableHandler.et_sess.ServerResponse.json()

In [99]:
jd['Tables'][0]

{'CharacterConfidence': 98.4,
 'LayoutConfidence': 77.09,
 'Page': 1,
 'TableConfidence': {'0': {'0': 99.6,
   '1': 99.38,
   '2': 99.7,
   '3': 99.85,
   '4': 99.94,
   '5': 99.87,
   '6': 99.45,
   '7': 99.59},
  '1': {'0': 99.81,
   '1': 99.16,
   '2': 99.25,
   '3': 0.0,
   '4': 0.0,
   '5': 0.0,
   '6': 0.0,
   '7': 0.0},
  '10': {'0': 99.81,
   '1': 0.0,
   '2': 96.98,
   '3': 0.0,
   '4': 0.0,
   '5': 0.0,
   '6': 0.0,
   '7': 0.0},
  '11': {'0': 99.59,
   '1': 0.0,
   '2': 99.87,
   '3': 98.65,
   '4': 99.79,
   '5': 0.0,
   '6': 99.77,
   '7': 0.0},
  '12': {'0': 99.53,
   '1': 0.0,
   '2': 98.22,
   '3': 0.0,
   '4': 0.0,
   '5': 0.0,
   '6': 0.0,
   '7': 0.0},
  '13': {'0': 99.79,
   '1': 99.81,
   '2': 99.5,
   '3': 91.7,
   '4': 0.0,
   '5': 0.0,
   '6': 0.0,
   '7': 0.0},
  '14': {'0': 99.93,
   '1': 99.8,
   '2': 83.17,
   '3': 92.12,
   '4': 99.82,
   '5': 99.87,
   '6': 99.84,
   '7': 0.0},
  '15': {'0': 99.72,
   '1': 99.92,
   '2': 69.36,
   '3': 98.17,
   '4': 99.8,

In [100]:
df = table_data.copy()

In [101]:
df[0]

,0,1,2,3,4,5,6,7
0,Course,Code,Course Title,Unit Attempted,Unit Gained,Grade,Grade Point,Remarks Code
1,Semester,1 of,2012-2013,,,,,
2,MATH,1110,Advanced Mathematics I (### I),3.00,3.00,B,9.00,
3,LANG,1011,Chinese I (A#+ I),3.00,3.00,B,9.00,
4,ENG,1011,English I (K#RX I),3.00,3.00,B-,8.10,
5,I.T.,1010,Information Technology,3.00,3.00,C+,6.90,
6,MATH,1070,Linear Algebra I (For FM Students) I),3.00,3.00,B+,9.90,
7,P.E.,1010,Physical Education/Healthy Living,3.00,3.00,B,9.00,
8,WPE,1010,University Life,1.00,1.00,C,2.00,
9,Semester,,Total BAPTIST,19.00,19.00,,51.90,


In [ ]:
"""
processd data, a list:
    1. table df data
    2. which page
    3. append confidence to each row
"""

In [110]:
df[0].to_csv(paths['output_dir']+'\\test-1.csv',index=False)

In [111]:
test_df1 = pd.read_csv(paths['output_dir']+'\\test-1.csv')

In [112]:
test_df1

,0,1,2,3,4,5,6,7
0,Course,Code,Course Title,Unit Attempted,Unit Gained,Grade,Grade Point,Remarks Code
1,Semester,1 of,2012-2013,NaN,NaN,NaN,NaN,NaN
2,MATH,1110,Advanced Mathematics I (### I),3.00,3.00,B,9.00,NaN
3,LANG,1011,Chinese I (A#+ I),3.00,3.00,B,9.00,NaN
4,ENG,1011,English I (K#RX I),3.00,3.00,B-,8.10,NaN
5,I.T.,1010,Information Technology,3.00,3.00,C+,6.90,NaN
6,MATH,1070,Linear Algebra I (For FM Students) I),3.00,3.00,B+,9.90,NaN
7,P.E.,1010,Physical Education/Healthy Living,3.00,3.00,B,9.00,NaN
8,WPE,1010,University Life,1.00,1.00,C,2.00,NaN
9,Semester,NaN,Total BAPTIST,19.00,19.00,NaN,51.90,NaN


In [114]:
def get_cell_confidence(i, r, c, server_response):
    """
    i: index of the table
    """
    return server_response['Tables'][i]['TableConfidence'][str(r)][str(c)]
get_cell_confidence(0,0,0,jd)

99.6

In [138]:
def table_confidence_check(threshold, i, server_response):
    """
    return a list of coor that has confidence below the threshold
    """
    result = [] # defective cells
    rate = 0
    valid_cells_count = 0 # non-zero confidence
    confidences = server_response['Tables'][i]['TableConfidence']
    for r in confidences:
        for c in confidences[r]:
            cell_conf = get_cell_confidence(i, int(r), int(c), server_response)
            if cell_conf > 0:
                valid_cells_count+=1
                if cell_conf < threshold:
                    result.append((int(r),int(c)))
    acceptance = 1 - len(result)/valid_cells_count
    return sorted(result), acceptance
table_confidence_check(98, 0, jd)

([(2, 2),
  (3, 2),
  (4, 2),
  (9, 2),
  (10, 2),
  (13, 3),
  (14, 2),
  (14, 3),
  (15, 2),
  (16, 2),
  (17, 2)],
 0.9219858156028369)

In [234]:
def read_response(student):
    paths = compose_paths(student)
    tables = []  # dfs
    server_response = None  # json
    outdir = paths['output_dir']
    for f in os.listdir(outdir):
        if f.split('.')[1]=='csv':
            t = pd.read_csv(os.path.join(outdir,f))
            tables.append(t)
        elif f.split('.')[1]=='json':
            with open(os.path.join(outdir,f)) as json_file:
                server_response = json.load(json_file)
    return tables, server_response
def compose_paths(student):
    paths = {}
    paths['input_file_path'] = os.path.join(BASE_DIR, "sample_output", student.name,
                                            f'{student.name}-raw-transcripts.pdf')
    paths['output_dir'] = os.path.join(BASE_DIR, "sample_output", student.name)
    return paths

In [238]:
dfs, server_response = read_response(liu)
table_coors = server_response['Tables'][i]['TableCoordinates']
columns, rows = dfs[i].shape
x2,y2 = 0, 0
for i in range(1, rows):
    if table_coors['0'][str(i-1)]:
        x2 = table_coors['0'][str(i-1)][2]
        break
for i in range(1, columns):
    if table_coors[str(i-1)]['0']:
        y2 = table_coors[str(i-1)]['0'][3]
        break
print(x2,y2)

0.0997 0.2976


In [185]:
def get_table_coor(i, server_response, dfs):
    """
    "{row#: {col#: <list(x1,y1,x2,y2)>}}"
    """
    table_coors = server_response['Tables'][i]['TableCoordinates']
    x1,y1 = table_coors['0']['0'][:2]
    x2,y2 = 0, 0
    columns, rows = dfs[i].shape
    for 
    
    return table_coors['0']['0'][:2]+[table_coors['0'][str(shape[1]-1)][2], table_coors[str(shape[0]-1)]['0'][3]]
get_table_coor(0, jd, df)

[0.071, 0.2738, 0.9686, 0.6722]

In [186]:
def extract_image_from_pdf_and_dump(input_pdf_path, output_path):
    doc = fitz.open(input_pdf_path)
    for i in range(len(doc)):
        for img in doc.get_page_images(i):
            xref = img[0]
            pix = fitz.Pixmap(doc, xref)
            if pix.n < 5:       # this is GRAY or RGB
                pix.save(output_path+"\\p%s.png" % (i))
            else:               # CMYK: convert to RGB first
                pix1 = fitz.Pixmap(fitz.csRGB, pix)
                pix1.save(output_path+"\\p%s.png" % (i))
                pix1 = None
            pix = None

In [210]:
def draw_rectangle_on_image_and_dump(base_dir, page_idx, table_idx, coors):
    input_image_path = os.path.join(base_dir, f"p{page_idx}.png")
    output_image_path = os.path.join(base_dir, f"p{page_idx}-t{table_idx}.png")
    image = Image.open(input_image_path)
    width, height = image.size
    draw = ImageDraw.Draw(image)
    draw.rectangle((coors[0]*width, coors[1]*height, coors[2]*width, coors[3]*height),outline =128, width=5)
#     image.show()
    print(f'compressing p{page_idx}-t{table_idx}.png ...')
    image = image.resize((int(width*0.7),int(height*0.7)),Image.ANTIALIAS)
    image.save(output_image_path,quality=70)
# draw_rectangle()

In [200]:
def get_table_border_images(input_pdf_path, output_path, server_response, dfs):
    """
    0. read pdf and extract iamges
    1. generate base image (on pages)
    2. draw rectangles (on tables)
    3. save
    input_pdf_path: input pdf path
    """
    extract_image_from_pdf_and_dump(input_pdf_path, output_path)
    pages_number = server_response['Pages']
    for i, table in enumerate(server_response['Tables']):
        coors = get_table_coor(i, server_response, dfs)
        draw_rectangle_on_image_and_dump(output_path, table["Page"]-1, i, coors)

In [211]:
get_table_border_images(paths['input_file_path'], paths['output_dir'], jd, df)

compressing p0-t0.png ...
compressing p0-t1.png ...
compressing p0-t2.png ...
compressing p1-t3.png ...
compressing p1-t4.png ...
compressing p1-t5.png ...
compressing p2-t6.png ...
compressing p2-t7.png ...
compressing p2-t8.png ...


In [221]:
ExtractTableHandler.prepare_transcript(deng)

sending pdf to ExtracTable...


C:\Projects\GC_beta\venv\lib\site-packages\ExtractTable\parsers.py:43: UserWarning: Check the status using the JobId after 30 seconds
  warnings.warn(_msg_)



[Info]: Waiting to retrieve the output; JobId: fe61eb906f2e5a9d2ed5346c4b74314d9cf9e3d5ea9137f81da0a2a5bede2f45
preparing images and tables...
compressing p0-t0.png ...
compressing p0-t1.png ...
compressing p0-t2.png ...
compressing p1-t3.png ...
compressing p1-t4.png ...
compressing p1-t5.png ...
compressing p2-t6.png ...
compressing p2-t7.png ...
compressing p2-t8.png ...


True

In [222]:
liu = Student.objects.get(id='618aec19396e535e251580c7')

In [302]:
from transcript_handler.handlers import ExtractTableHandler
ExtractTableHandler.prepare_transcript(deng,False)

checking data...
C:\Projects\GC_beta\GC_beta\sample_output\Deng\server_response.json True
Data found. Reading from dumped files instead...
reading server response...
reading table-0.csv
reading table-1.csv
reading table-2.csv
reading table-3.csv
reading table-4.csv
reading table-5.csv
reading table-6.csv
reading table-7.csv
reading table-8.csv
preparing images and tables...
preparing table 0
table 0 shape: 25 8
compressing p0-t0.png ...
preparing table 1
table 1 shape: 5 2
compressing p0-t1.png ...
preparing table 2
table 2 shape: 3 2
compressing p0-t2.png ...
preparing table 3
table 3 shape: 24 8
compressing p1-t3.png ...
preparing table 4
table 4 shape: 5 2
compressing p1-t4.png ...
preparing table 5
table 5 shape: 3 2
compressing p1-t5.png ...
preparing table 6
table 6 shape: 26 8
compressing p2-t6.png ...
preparing table 7
table 7 shape: 5 2
compressing p2-t7.png ...
preparing table 8
table 8 shape: 3 2
compressing p2-t8.png ...


True

In [263]:
tables[0].shape

(25, 9)

In [262]:
wrong_tables, server_response = ExtractTableHandler._read_response(deng)

In [264]:
tables[0]

,Unnamed: 0,0,1,2,3,4,5,6,7
0,0,Course,Code,Course Title,Unit Attempted,Unit Gained,Grade,Grade Point,Remarks Code
1,1,Semester,1 of,2012-2013,NaN,NaN,NaN,NaN,NaN
2,2,MATH,1110,Advanced Mathematics I (### I),3.00,3.00,B,9.00,NaN
3,3,LANG,1011,Chinese I (A#+ I),3.00,3.00,B,9.00,NaN
4,4,ENG,1011,English I (K#RX I),3.00,3.00,B-,8.10,NaN
5,5,I.T.,1010,Information Technology,3.00,3.00,C+,6.90,NaN
6,6,MATH,1070,Linear Algebra I (For FM Students) I),3.00,3.00,B+,9.90,NaN
7,7,P.E.,1010,Physical Education/Healthy Living,3.00,3.00,B,9.00,NaN
8,8,WPE,1010,University Life,1.00,1.00,C,2.00,NaN
9,9,Semester,NaN,Total BAPTIST,19.00,19.00,NaN,51.90,NaN


In [266]:
from transcript_handler.handlers import ExtractTableHandler

paths = ExtractTableHandler._compose_paths(deng)
table_data = ExtractTableHandler.et_sess.process_file(filepath=paths['input_file_path'], pages="all",
                                       output_format="df")
server_response = ExtractTableHandler.et_sess.ServerResponse.json()

C:\Projects\GC_beta\venv\lib\site-packages\ExtractTable\parsers.py:43: UserWarning: Check the status using the JobId after 30 seconds
  warnings.warn(_msg_)



[Info]: Waiting to retrieve the output; JobId: f7d29097ac8e292f6916ca6c5b33e82ace1d5e7ae8935d6997a9e1440e912ea5


In [267]:
table_data[0].shape

(25, 8)

In [268]:
table_data[0]

,0,1,2,3,4,5,6,7
0,Course,Code,Course Title,Unit Attempted,Unit Gained,Grade,Grade Point,Remarks Code
1,Semester,1 of,2012-2013,,,,,
2,MATH,1110,Advanced Mathematics I (### I),3.00,3.00,B,9.00,
3,LANG,1011,Chinese I (A#+ I),3.00,3.00,B,9.00,
4,ENG,1011,English I (K#RX I),3.00,3.00,B-,8.10,
5,I.T.,1010,Information Technology,3.00,3.00,C+,6.90,
6,MATH,1070,Linear Algebra I (For FM Students) I),3.00,3.00,B+,9.90,
7,P.E.,1010,Physical Education/Healthy Living,3.00,3.00,B,9.00,
8,WPE,1010,University Life,1.00,1.00,C,2.00,
9,Semester,,Total BAPTIST,19.00,19.00,,51.90,


In [270]:
table_data[0].to_csv(os.path.join(paths['output_dir'],'test.csv'), index=False)

In [269]:
paths['output_dir'],'test.csv'

('C:\\Projects\\GC_beta\\GC_beta\\sample_output\\Deng', 'test.csv')

In [294]:
paths = ExtractTableHandler._compose_paths(liu)
test_table = pd.read_csv(os.path.join(paths['output_dir'],'table-0.csv'))

In [295]:
test_table.shape

(29, 8)

In [275]:
len(server_response['Tables'])

9

In [297]:
from transcript_handler.handlers import ExtractTableHandler
wrong_tables, server_response = ExtractTableHandler._read_response(liu)

reading server response...
read table-0.csv
read table-1.csv
read table-2.csv
read table-3.csv
read table-4.csv
read table-5.csv
read table-6.csv
read table-7.csv
read table-8.csv


In [298]:
wrong_tables[0]

,0,1,2,3,4,5,6,7
0,Course,Code,Course Title,Unit Attempted,Unit Gained,Grade,Grade Point,Remarks Code
1,Semester,1 of,2015-2016,NaN,NaN,NaN,NaN,NaN
2,GCLA,1903,English I it I),3.00,3.00,A-,11.01,NaN
3,WPEX,1003,Experiential Development,1.00,1.00,B,3.00,NaN
4,GCIT,1003,IT for Success in Everyday Life and Work (B ***),3.00,3.00,A-,11.01,NaN
5,ECON,2013,Principles of Microeconomies,3.00,3.00,A-,11.01,NaN
6,GDFL,1033,Spanish I,3.00,3.00,A,12.00,NaN
7,GCPE,1203,Table Tennis (FR#),1.00,1.00,B,3.00,NaN
8,GCCH,1003,University Chinese it # *),3.00,3.00,C+,6.99,NaN
9,Semester,Total,RONG,NaN,17.00,NaN,58.02,NaN
